In [42]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout


In [43]:
data = []
labels = []
classes = 43
current_path = os.getcwd()
 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        image = Image.open(path + '\\'+ a)
        image = image.resize((30,30))
        image = np.array(image)
        data.append(image)
        labels.append(i)


In [44]:
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)
#Splits the training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print('Shape of input arrays :')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Converts the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

(39209, 30, 30, 3) (39209,)
(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [45]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("trafficsign_model.h5")

Train on 31367 samples, validate on 7842 samples
Epoch 1/15
31367/31367 [==============================] - 11s 347us/step - loss: 2.0593 - accuracy: 0.4646 - val_loss: 0.6819 - val_accuracy: 0.8226
Epoch 2/15
31367/31367 [==============================] - 9s 295us/step - loss: 0.7373 - accuracy: 0.7830 - val_loss: 0.2503 - val_accuracy: 0.9287
Epoch 3/15
31367/31367 [==============================] - 9s 294us/step - loss: 0.4736 - accuracy: 0.8590 - val_loss: 0.1622 - val_accuracy: 0.9573
Epoch 4/15
31367/31367 [==============================] - 10s 305us/step - loss: 0.3957 - accuracy: 0.8830 - val_loss: 0.1525 - val_accuracy: 0.9570
Epoch 5/15
31367/31367 [==============================] - 10s 317us/step - loss: 0.3328 - accuracy: 0.9024 - val_loss: 0.1288 - val_accuracy: 0.9667
Epoch 6/15
31367/31367 [==============================] - 11s 340us/step - loss: 0.3135 - accuracy: 0.9097 - val_loss: 0.1010 - val_accuracy: 0.9739
Epoch 7/15
31367/31367 [==============================] - 1

In [48]:
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

pred = model.predict_classes(X_test)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

0.9463182897862232


In [49]:
model.save('traffic_classifier.h5')

In [55]:
new_model = tf.keras.models.load_model(filepath='traffic_classifier.h5')
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = tflite_converter.convert()
open("tf_lite_model.tflite", "wb").write(tflite_model)

972172